In [1]:
import os

In [2]:
%pwd

'e:\\Hate-Speech-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Hate-Speech-Classification'

In [5]:
# Define the entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    Transformed_filename: Path
    Data_dir: str
    Id: str
    Axis: int
    Inplace: bool
    Drop_Columns: list
    Class: str
    Label: str
    Tweet: str

    


In [6]:
# Configuration manager
from Hate_Speech_Classification.constrants import * # Import Everything
from Hate_Speech_Classification.utils.common import read_yaml,create_directories 

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,  # Return Box Type  # Ctrl+click to check the file path
            params_filepath=PARAMS_FILE_PATH):

            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)

            # From common.py
            create_directories([self.config.artifacts_root]) # I can call using the key name using Box Type

    def get_data_Transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir]) 

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            Transformed_filename=config.Transformed_filename,
            Data_dir=config.Data_dir,
            Id=config.Id,
            Axis=config.Axis,
            Inplace=config.Inplace,
            Drop_Columns=config.Drop_Columns,
            Class=config.Class,
            Label=config.Label,
            Tweet=config.Tweet


        )

        return data_transformation_config

In [8]:
import os
import re
import sys
import string
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from Hate_Speech_Classification.Logging import logging 
from Hate_Speech_Classification.Exception import CustomException

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config



    def imbalance_data_cleaning(self):

        try:
            logging.info("Entered into the imbalance_data_cleaning function")
            imbalance_data=pd.read_csv("artifacts\data_ingestion\imbalanced_data.csv")
            imbalance_data.drop(self.config.Id,axis=self.config.Axis , inplace = self.config.Inplace)
            logging.info(f"Exited the imbalance data_cleaning function and returned imbalance data {imbalance_data}")
            return imbalance_data 
        except Exception as e:
            raise CustomException(e,sys) from e 



    def raw_data_cleaning(self):

        try:
            logging.info("Entered into the raw_data_cleaning function")
            raw_data = pd.read_csv("artifacts/data_ingestion/raw_data.csv")
            raw_data.drop(self.config.Drop_Columns,axis = self.config.Axis,
            inplace = self.config.Inplace)


            raw_data[raw_data[self.config.Class]==0][self.config.Class]=1

            # replace the value of 0 to 1
            raw_data[self.config.Class].replace({0:1},inplace=True)

            # Let's replace the value of 2 to 0.
            raw_data[self.config.Class].replace({2:0}, inplace = True)

            # Let's change the name of the 'class' to label
            raw_data.rename(columns={self.config.Class:self.config.Label},inplace =True)
            logging.info(f"Exited the raw_data_cleaning function and returned the raw_data {raw_data}")
            return raw_data

        except Exception as e:
            raise CustomException(e,sys) from e



    def concat_dataframe(self):

        try:
            logging.info("Entered into the concat_dataframe function")
            # Let's concatinate both the data into a single data frame.
            frame = [self.raw_data_cleaning(), self.imbalance_data_cleaning()]
            df = pd.concat(frame)
            print(df.head())
            logging.info(f"returned the concatinated dataframe {df}")
            return df

        except Exception as e:
            raise CustomException(e, sys) from e



    def concat_data_cleaning(self, words):

        try:
            logging.info("Entered into the concat_data_cleaning function")
            stemmer = nltk.SnowballStemmer("english")
            stopword = set(stopwords.words('english'))
            
            words = str(words).lower()
            words = re.sub('\[.*?\]', '', words)
            words = re.sub('https?://\S+|www\.\S+', '', words)
            words = re.sub('<.*?>+', '', words)
            words = re.sub('[%s]' % re.escape(string.punctuation), '', words)
            words = re.sub('\n', '', words)
            words = re.sub('\w*\d\w*', '', words)

            # Remove stopwords and apply stemming
            words = [word for word in words.split() if word not in stopword]
            words = " ".join([stemmer.stem(word) for word in words])

            logging.info("Exited the concat_data_cleaning function")
            return words
        except Exception as e:
            raise CustomException(e, sys) from e



    def initiate_data_transformation(self):
        try:
            logging.info("Entered the initiate_data_transformation method of Data transformation class")
            
            # Cleaning and transforming data
            df = self.concat_dataframe()
            df[self.config.Tweet] = df[self.config.Tweet].apply(self.concat_data_cleaning)

            # Save the transformed data
            os.makedirs(self.config.root_dir, exist_ok=True)
            df.to_csv(self.config.Transformed_filename, index=False, header=True)

            # Return the DataTransformationConfig object
            data_transformation_artifact = DataTransformationConfig(
                root_dir=self.config.root_dir,
                Transformed_filename=self.config.Transformed_filename,
                Data_dir=self.config.Data_dir,
                Id=self.config.Id,
                Axis=self.config.Axis,
                Inplace=self.config.Inplace,
                Drop_Columns=self.config.Drop_Columns,
                Class=self.config.Class,
                Label=self.config.Label,
                Tweet=self.config.Tweet
            )
            logging.info("Returning the DataTransformationArtifacts")
            return data_transformation_artifact
        except Exception as e:
            raise CustomException(e, sys) from e

In [11]:
# Pipeline
from Hate_Speech_Classification.Logging import logging 
from Hate_Speech_Classification.Exception import CustomException
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_Transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation_artifact = data_transformation.initiate_data_transformation()

except Exception as e:
    raise CustomException(e, sys) from e

C:\Users\HP\AppData\Local\Temp\ipykernel_6508\250299221.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data[raw_data[self.config.Class]==0][self.config.Class]=1


   label                                              tweet
0      0  !!! RT @mayasolovely: As a woman you shouldn't...
1      1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2      1  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3      1  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4      1  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
